In [2]:
import pandas as pd
import sqlite3

In [6]:
orders=pd.read_csv("orders.csv")
users=pd.read_json("users.json")

In [7]:
orders.head()
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [9]:
conn = sqlite3.connect("restaurants.db")
with open("restaurants.sql","r") as f:
  sql_script=f.read()
  conn.executescript(sql_script)
  _restaurants=pd.read_sql("SELECT*FROM restaurants",conn)

In [11]:
_restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [13]:
df=orders.merge(users,on="user_id",how="left")
df=df.merge(_restaurants,on="restaurant_id",how="left")

In [14]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   order_id           10000 non-null  int64  
 1   user_id            10000 non-null  int64  
 2   restaurant_id      10000 non-null  int64  
 3   order_date         10000 non-null  object 
 4   total_amount       10000 non-null  float64
 5   restaurant_name_x  10000 non-null  object 
 6   name               10000 non-null  object 
 7   city               10000 non-null  object 
 8   membership         10000 non-null  object 
 9   restaurant_name_y  10000 non-null  object 
 10  cuisine            10000 non-null  object 
 11  rating             10000 non-null  float64
dtypes: float64(2), int64(3), object(7)
memory usage: 937.6+ KB


In [15]:
df.to_csv("final_food_delivery_dataset.csv",index=False)

In [16]:
df[df['membership']=='Gold']\
.groupby('city')['total_amount']\
.sum()\
.sort_values(ascending=False)

,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [18]:
df.groupby('cuisine')['total_amount'] \
.mean() \
.sort_values(ascending=False)

,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [19]:
df['order_date'] = pd.to_datetime(df['order_date'])
df.groupby(df['order_date'].dt.month)['order_id'].count()

/tmp/ipython-input-3509095697.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['order_date'] = pd.to_datetime(df['order_date'])


,order_id
order_date,
1,831
2,785
3,903
4,812
5,844
6,784
7,859
8,851
9,812


In [20]:
df.groupby('membership')['total_amount'].mean()

,total_amount
membership,
Gold,797.145556
Regular,805.158434


In [21]:
df[df['membership'] == 'Gold'] \
.groupby('city')['total_amount'] \
.mean() \
.sort_values(ascending=False)

,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


In [22]:
restaurant_count = df.groupby('cuisine')['restaurant_id'].nunique()





revenue = df.groupby('cuisine')['total_amount'].sum()





summary = pd.concat([restaurant_count, revenue], axis=1)
summary.columns = ['restaurant_count', 'revenue']
summary.sort_values(by=['restaurant_count', 'revenue'], ascending=[True, False])

,restaurant_count,revenue
cuisine,,
Chinese,120,1930504.65
Italian,126,2024203.80
Indian,126,1971412.58
Mexican,128,2085503.09


In [23]:
total_orders = df.shape[0]
gold_orders = df[df['membership'] == 'Gold'].shape[0]

percentage = round((gold_orders / total_orders) * 100)
percentage

50

In [25]:
restaurant_stats = df.groupby('restaurant_name_y').agg(
    total_orders=('order_id', 'count'),
    avg_order_value=('total_amount', 'mean')
)

restaurant_stats[restaurant_stats['total_orders'] < 20] \
.sort_values(by='avg_order_value', ascending=False)

,total_orders,avg_order_value
restaurant_name_y,,
Restaurant_294,13,1040.222308
Restaurant_262,18,1029.473333
Restaurant_77,12,1029.180833
Restaurant_193,15,1026.306667
Restaurant_7,16,1002.140625
...,...,...
Restaurant_184,19,621.828947
Restaurant_498,18,596.815556
Restaurant_192,14,589.972857


In [26]:

df.groupby(['membership', 'cuisine'])['total_amount'] \
.sum() \
.sort_values(ascending=False)

membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [27]:

df['order_date'] = pd.to_datetime(df['order_date'])

df['quarter'] = df['order_date'].dt.to_period('Q')

df.groupby('quarter')['total_amount'] \
.sum() \
.sort_values(ascending=False)

,total_amount
quarter,
2023Q3,2037385.10
2023Q4,2018263.66
2023Q1,1993425.14
2023Q2,1945348.72
2024Q1,17201.50


In [29]:
df[df['membership']=='Gold']['order_id'].nunique()

4987

In [30]:
round(
    df[df['city'] == 'Hyderabad']['total_amount'].sum()
)

1889367

In [31]:
df['user_id'].nunique()

2883

In [33]:
round(
    df[df['membership'] == 'Gold']['total_amount'].mean(),
    2
)

np.float64(797.15)

In [34]:
df[df['rating'] >= 4.5]['order_id'].nunique()

3374

In [35]:
gold_city_revenue = (
    df[df['membership'] == 'Gold']
    .groupby('city')['total_amount']
    .sum()
    .sort_values(ascending=False)
)

gold_city_revenue
top_city = gold_city_revenue.index[0]

df[
    (df['membership'] == 'Gold') &
    (df['city'] == top_city)
]['order_id'].nunique()

1337